In [543]:
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import pandas as pd

In [544]:
try:
    conexion = psycopg2.connect(
        database = "forq2",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"
    )
except OperationalError as e:
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print("Contraseña es errónea")
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print("Error de conexión")
    else:
        print(f"Ocurrió el error {e}") 

cursor = conexion.cursor()

In [545]:
df_f = pd.read_csv("datos/api_foursquare.csv", index_col = 0)
df_muni = pd.read_csv("datos/municipios.csv", index_col = 0)
df_ae = pd.read_csv("datos/df_aemet_final.csv", index_col = 0)

In [546]:
df_ae.reset_index(inplace=True)
df_ae.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001


In [547]:
df_ae[['localizacion', 'id']] = df_ae['localizacion_id'].str.split('-id', expand=True)
df_ae[['fecha', 'hora']] = df_ae['fecha'].str.split(' ', expand=True)

In [548]:
df_ae.head(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id,localizacion,id,hora
0,2024-08-29,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001,acebeda-la,28001,12:00:00
1,2024-08-29,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001,acebeda-la,28001,13:00:00


In [549]:
# df_ae = df_ae.drop(columns = "level_0")
# df_ae = df_ae.drop(columns = "index")

In [550]:
df_ae.rename(columns={'localizacion': 'id_municipio'}, inplace=True)
df_ae.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id,id_municipio,id,hora
0,2024-08-29,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001,acebeda-la,28001,12:00:00


In [551]:
df_ae = df_ae.drop(columns = "localizacion_id")
df_ae = df_ae.drop(columns = "prob._precip.__(%)")
df_ae = df_ae.drop(columns = "id") 

In [552]:
df_f.rename(columns={'municipio': 'municipio_id'}, inplace=True)


In [553]:
df_f['latitude'] = df_f['latitude'].astype(float) 
df_f['longitude'] = df_f['longitude'].astype(float)

In [554]:
df_f.head(1)

,municipio_id,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341


In [555]:
df_muni = df_muni.T
df_muni.reset_index(inplace=True)
df_muni

,index,0,1,2
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo
4,alcala-de-henares,40.4818396,-3.3644973,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.3364465,-3.2355624,Villar del Olmo
172,villarejo-de-salvanes,40.1683582,-3.2738764,Villarejo de Salvanés
173,villaviciosa-de-odon,40.3573787,-3.9002334,Villaviciosa de Odón
174,villavieja-del-lozoya,41.0063622,-3.6716291,Villavieja del Lozoya


In [556]:
df_muni.rename(columns={'index': 'id_municipio', 0: 'latitud', 1:'longitud',2:'nombre municipio'}, inplace=True)

In [557]:
df_muni['latitud'] = df_muni['latitud'].astype(float)
df_muni['longitud'] = df_muni['longitud'].astype(float)

In [558]:
set_aemet = set(df_ae['id_municipio'])
set_municipios = set(df_muni['id_municipio'])

In [559]:
nombres_limpiar = set_aemet-set_municipios

In [560]:
nombres_limpiar

{'cabrera-la', 'leganes', 'molinos-los'}

In [561]:
df_ae = df_ae.loc[df_ae['id_municipio'] != 'cabrera-la']
df_ae = df_ae.loc[df_ae['id_municipio'] != 'leganes']
df_ae = df_ae.loc[df_ae['id_municipio'] != 'molinos-los']

In [562]:
# nuevo_df_ae = df_ae[['municipio_id', 'fecha', 'cielo']]

In [563]:
lista_tuplas_municipios = []
for fila in df_muni.values:
    lista_tuplas_municipios.append(tuple(fila))


In [564]:
query_creacion_municipios = '''
    create table if not exists municipios(
        id_municipio VARCHAR(100) primary key, 
        latitud INT,
        longitud INT,
        nombre VARCHAR(100)
        ); '''

cursor.execute(query_creacion_municipios)
conexion.commit()

In [565]:
# query_insercion_municipios = '''
#     insert into municipios (id_municipio, latitud, longitud, nombre)
#     values
#     (%s, %s, %s, %s)''' 

# cursor.executemany(query_insercion_municipios, lista_tuplas_municipios) 
# conexion.commit()

In [566]:
df_f.head(2)

,municipio_id,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773


In [567]:
df_f= df_f.drop(columns = "fsq_id")

In [568]:
df_f.head(2)

,municipio_id,category,closed_bucket,distance,link,name,address,latitude,longitude
0,ajalvir,Park,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341
1,ajalvir,Park,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773


In [569]:
lista_tuplas_f = []
for fila in df_f.values:
    lista_tuplas_f.append(tuple(fila))


In [570]:
len(lista_tuplas_f[0])

9

In [571]:
query_creacion_locales = '''
    create table if not exists locales (
        id_local SERIAL primary key,
        id_municipio VARCHAR(100)
            references municipios(id_municipio)
            on update cascade,
        categoria CHAR(50),
        estado CHAR(50),
        distancia INT,
        link VARCHAR(100),
        nombre VARCHAR(100),
        direccion VARCHAR(300),
        latitud DECIMAL,
        longitud DECIMAL
    );'''

cursor.execute(query_creacion_locales)
conexion.commit()

In [572]:
query_insercion_locales = '''
    insert into locales (id_municipio, categoria,estado, distancia, link, nombre, direccion, latitud, longitud)
    values
    (%s, %s, %s, %s, %s, %s, %s, %s, %s)''' 

cursor.executemany(query_insercion_locales, lista_tuplas_f) 
conexion.commit()

In [581]:
df_ae.head()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,id_municipio,hora
0,2024-08-29,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7,acebeda-la,12:00:00
1,2024-08-29,Nubes altas,24,24,29.0,0.0,0,57,0,75,SE,12,acebeda-la,13:00:00
2,2024-08-29,Muy nuboso,23,23,33.0,0.0,0,59,0,80,S,14,acebeda-la,14:00:00
3,2024-08-29,Nubes altas,23,23,35.0,0.0,0,55,0,80,S,17,acebeda-la,15:00:00
4,2024-08-29,Nubes altas,21,21,34.0,0.0,0,65,0,80,S,20,acebeda-la,16:00:00


In [582]:
lista_tuplas_ae = []
for fila in df_ae.values:
    lista_tuplas_ae.append(tuple(fila))

In [583]:
len(lista_tuplas_ae[0])

14

In [584]:
query_creacion_ae = '''
    create table if not exists clima (
        id_clima SERIAL primary key,
        fecha DATE,
        cielo VARCHAR(50),
        temperatura DECIMAL,
        sensacion DECIMAL,
        racha_max DECIMAL,
        precipitacion DECIMAL,
        nieve DECIMAL,
        humedad_relativa DECIMAL,
        prob_nieve DECIMAL,
        prob_tormenta DECIMAL,
        direccion_viento VARCHAR(10),
        velocidad_viento DECIMAL,
        municipio_id VARCHAR(100)
            references municipios(id_municipio)
            on update cascade,
        hora VARCHAR (20)
    );'''

cursor.execute(query_creacion_ae)
conexion.commit()

In [576]:
df_ae.head(2)
df_ae= df_ae.drop(columns = "avisos")

In [577]:
df_ae.head(1)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._de_nieve_(%),prob._de_tormenta_(%),dirección_viento,velocidad_del_viento,id_municipio,hora
0,2024-08-29,Nubes altas,21,21,25.0,0.0,0,69,0,75,S,7,acebeda-la,12:00:00


In [585]:
query_insercion_ae = '''
    insert into clima (fecha, cielo, temperatura, sensacion, racha_max, precipitacion, nieve, humedad_relativa,prob_nieve,prob_tormenta,direccion_viento,velocidad_viento,municipio_id,hora)
    values
    (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)'''

cursor.executemany(query_insercion_ae, lista_tuplas_ae)
conexion.commit()